# Crypto Token Correlation Heatmep

### Fetch Tokens Data from Coingecko

In [19]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd

load_dotenv()  # Load environment variables from .env

api_key = os.getenv('GECKO_API')  # Get API key from .env
coin_id = 'bitcoin'
vs_currency = 'usd'
days = '30'

url = f'https://pro-api.coingecko.com/api/v3/coins/{coin_id}/market_chart'
params = {
    'vs_currency': vs_currency,
    'days': days
}
headers = {
    'accept': 'application/json',
    'x-cg-pro-api-key': api_key
}

response = requests.get(url, headers=headers, params=params)

### Convert Result into a DataFrame

In [20]:
data = response.json()

# Convert 'prices' to DataFrame
df = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

print(df.head())

                timestamp          price
0 2025-07-06 12:03:51.499  108079.788276
1 2025-07-06 13:03:43.969  108226.778238
2 2025-07-06 14:04:04.610  108852.242818
3 2025-07-06 15:00:49.792  108881.002859
4 2025-07-06 16:03:42.914  108842.004636


### Create a Function

In [21]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd
import joblib

load_dotenv()  # Load environment variables from .env

api_key = os.getenv('GECKO_API')  # Get API key from .env
coin_id = 'bitcoin'
vs_currency = 'usd'
days = '30'

def fetch_token_price_history(token_ids, vs_currency='usd', days='30'):
    """
    Fetch historical price data for multiple tokens from CoinGecko Pro API.
    
    Args:
        token_ids (list): List of token ids (e.g., ['bitcoin', 'ethereum']).
        vs_currency (str): The target currency (default 'usd').
        days (str): Number of days to fetch (default '30').
    
    Returns:
        pd.DataFrame: DataFrame with timestamps as index and tokens as columns.
    """
    load_dotenv()
    api_key = os.getenv('GECKO_API')
    price_dfs = []
    for coin_id in token_ids:
        
        url = f'https://pro-api.coingecko.com/api/v3/coins/{coin_id}/market_chart'
        params = {
            'vs_currency': vs_currency,
            'days': days
        }
        headers = {
            'accept': 'application/json',
            'x-cg-pro-api-key': api_key
        }
        response = requests.get(url, headers=headers, params=params)
        data = response.json()
        df = pd.DataFrame(data['prices'], columns=['timestamp', coin_id])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        df = df.resample('1h').last()  # or '1D' for daily
        price_dfs.append(df)

    # Merge all dataframes on timestamp
    result_df = pd.concat(price_dfs, axis=1)
    return result_df

In [22]:
# Example: Define tokens directly for notebook use
tokens = ['bitcoin', 'ethereum', 'solana', 'ripple']  # Add more as needed

# Example: Get tokens from user input
# tokens = input("Enter token IDs separated by commas (e.g., bitcoin,ethereum,solana): ").split(",")
# tokens = [t.strip() for t in tokens]

df_prices = fetch_token_price_history(tokens, days='30')
print(df_prices.head(5000))

# Save df_prices using joblib
save_dir = 'C:/Users/FFFO CASHIER PT/OneDrive/Desktop/Hands-on Projects/database'
os.makedirs(save_dir, exist_ok=True)
joblib.dump(df_prices, os.path.join(save_dir, "df_prices.joblib"))

                           bitcoin     ethereum      solana    ripple
timestamp                                                            
2025-07-06 12:00:00  108079.788276  2514.979427  147.860826       NaN
2025-07-06 13:00:00  108226.778238  2520.355851  148.786268  2.273416
2025-07-06 14:00:00  108852.242818  2560.016103  151.526358  2.284491
2025-07-06 15:00:00  108881.002859  2558.079219  151.513413  2.270192
2025-07-06 16:00:00  108842.004636  2555.859603  151.277611  2.275761
...                            ...          ...         ...       ...
2025-08-05 08:00:00  113952.075023  3612.362157  166.097742  3.006206
2025-08-05 09:00:00  114292.621861  3635.046187  166.833115  3.036377
2025-08-05 10:00:00  114683.080592  3659.569446  169.087926  3.057521
2025-08-05 11:00:00  114783.830937  3675.375878  170.696358  3.070277
2025-08-05 12:00:00  114790.749301  3670.576064  170.791193  3.060353

[721 rows x 4 columns]


['C:/Users/FFFO CASHIER PT/OneDrive/Desktop/Hands-on Projects/database\\df_prices.joblib']

In [23]:

save_dir = '\Users\FFFO CASHIER PT\OneDrive\Desktop\Hands-on Projects/Database'
os.makedirs(save_dir, exist_ok=True)
joblib.dump(df_prices, os.path.join(save_dir, "df_prices.joblib"))

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 0-1: truncated \UXXXXXXXX escape (2952044808.py, line 1)